In [2]:
import numpy as np
import pandas as pd


In [3]:
import torch
from tqdm.notebook import tqdm


In [5]:
df = pd.read_csv("data/elec-even.csv")
df = df.sample(15000)

In [6]:
possible_labels = df.overall.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

df['category'] = df['overall'].map(label_dict)
df.category.value_counts()

3    3029
0    3024
4    3004
1    2996
2    2947
Name: category, dtype: int64

In [6]:
df.head(10)

,reviewText,overall,neg,neu,pos,comp,category
223811,Comfortable headset works great until it break...,3,0.043,0.791,0.166,0.8313,0
263550,these fans are the best hands down but these i...,2,0.250,0.717,0.034,-0.9556,1
85559,I was disappointed to find out that it was onl...,3,0.067,0.887,0.046,-0.1655,0
281778,"When I purchased it many years ago, it was exc...",5,0.000,0.801,0.199,0.7351,2
279838,Being able to quickly and easily mount and rem...,5,0.000,0.836,0.164,0.7269,2
220734,The only way I got the charger to work was to ...,1,0.000,0.869,0.131,0.6467,3
43359,Good price.... Good quality ....Arrived on ti...,5,0.088,0.669,0.243,0.5705,2
280778,These plugs are solidly constructed. I do wis...,4,0.000,0.713,0.287,0.9230,4
147796,Used 2 of these cameras for a small CCTV syste...,1,0.000,0.973,0.027,0.3716,3
13401,Worked well. I bought it so my husband and I ...,5,0.000,0.735,0.265,0.7351,2


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.category.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.category.values)



In [8]:
df['data_type'] = ['not_set']*df.shape[0]

In [9]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [10]:
df.groupby(['category', 'data_type']).count()


reviewText  overall   neg   neu   pos  comp
category data_type                                             
0        train            2570     2570  2570  2570  2570  2570
         val               454      454   454   454   454   454
1        train            2547     2547  2547  2547  2547  2547
         val               449      449   449   449   449   449
2        train            2505     2505  2505  2505  2505  2505
         val               442      442   442   442   442   442
3        train            2575     2575  2575  2575  2575  2575
         val               454      454   454   454   454   454
4        train            2553     2553  2553  2553  2553  2553
         val               451      451   451   451   451   451

In [11]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [12]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [13]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].reviewText.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].reviewText.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)

In [14]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [15]:
len(dataset_train)

12750

In [16]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased',
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)


In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)
epochs = 2

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

/mnt/Data/PycharmProjects/DM-i-AI-2022/sentiment-analysis/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [19]:
import numpy as np
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [20]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [21]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals



In [22]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_val,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_val)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_train)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/71 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 3.82 GiB total capacity; 2.88 GiB already allocated; 1.62 MiB free; 2.92 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [4]:
torch.cuda.empty_cache()
